# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.12.207.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService("davinci", model, apiKey, orgId);

Let's define a prompt outlining a dialogue chat bot.

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
Human: {{$human_input}}
ChatBot:";

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.7,
        TopP = 0.5,
    }
};

Register your semantic function

In [ ]:
var promptTemplate = new PromptTemplate(skPrompt, promptConfig, kernel);
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

Initialize your context

In [ ]:
var context = new ContextVariables();

var history = "";
context.Set("history", history);

Chat with the Bot

In [ ]:
var human_input = "Hi, I'm looking for book suggestions";
context.Set("human_input", human_input);

var bot_answer = await kernel.RunAsync(context, chatFunction);

Update the history with the output

In [ ]:
history += $"\nHuman: {human_input}\nMelody: {bot_answer}\n";
context.Update(history);

Console.WriteLine(context);


Human: Hi, I'm looking for book suggestions
Melody:  Hi there! What kind of books are you looking for?



Keep Chatting!

In [ ]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Set("human_input", input);

    // Process the user message and get an answer
    var answer = await kernel.RunAsync(context, chatFunction);

    // Append the new interaction to the chat history
    history += $"\nHuman: {input}\nMelody: {answer}\n"; context.Set("history", history);
    
    // Show the response
    Console.WriteLine(context);
};

In [ ]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

 The Landmark Thucydides: A Comprehensive Guide to the Peloponnesian War by Robert B. Strassler is a great book to learn about the history of Greece.


In [ ]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

 The Landmark Thucydides covers topics such as the causes of the Peloponnesian War, the strategies and tactics used by the Greeks and Persians, the roles of Athens and Sparta, and the impact of the war on the Greek world.


In [ ]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");

 Most people find the strategies and tactics used by the Greeks and Persians to be the most interesting topic in The Landmark Thucydides.


In [ ]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");

 Most people find the strategies and tactics used by the Greeks and Persians to be the most interesting topic in The Landmark Thucydides.
